/bin/bash: /global/homes/g/gzquse/.local/bin/pip: /opt/conda/bin/python3.8: bad interpreter: No such file or directory


In [1]:
import numpy as np
import scipy.stats
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import random_density_matrix
import pandas as pd

# Define the number of qubits
num_qubits = 4

# Function to create a QAOA-like state with hybrid unitaries
def generate_qaoa_state(num_qubits):
    qc = QuantumCircuit(num_qubits)
    for qubit in range(num_qubits):
        qc.h(qubit)
    for qubit in range(num_qubits - 1):
        qc.cx(qubit, qubit + 1)
    qc.measure_all()
    return qc

# Generate Haar-random state
haar_random_state = random_density_matrix(2**num_qubits, rank=2**num_qubits).data

# Simulate QAOA circuit
qaoa_circuit = generate_qaoa_state(num_qubits)
simulator = AerSimulator()
transpiled_qc = transpile(qaoa_circuit, simulator)
job = simulator.run(transpiled_qc, shots=10000)
qaoa_results = job.result().get_counts()

# Convert QAOA measurement results into a probability distribution
qaoa_probs = np.array([qaoa_results.get(format(i, f"0{num_qubits}b"), 0) for i in range(2**num_qubits)]) / 10000

# Generate Haar-random state probability distribution
haar_probs = np.real(np.diagonal(haar_random_state))

# Normalize Haar-random probabilities
haar_probs /= np.sum(haar_probs)

# Compute KL divergence
kl_divergence = scipy.stats.entropy(qaoa_probs + 1e-10, haar_probs + 1e-10)  # Add epsilon to avoid log(0)

# Compute trace distance
trace_distance = 0.5 * np.linalg.norm(qaoa_probs - haar_probs, ord=1)

# Display results
results_df = pd.DataFrame({
    "Metric": ["KL Divergence", "Trace Distance"],
    "Value": [kl_divergence, trace_distance],
})

print(results_df)


ModuleNotFoundError: No module named 'qiskit_aer'